# **The fifth in-class-exercise (40 points in total, 4/18/2023)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

(7) Word2Vec

(8) BERT

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [ ]:
# Write your code here
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [ ]:
with open("stsa-train.txt") as txtf:
    list_1 = [line.rstrip('\n') for line in txtf]
    
labels_1 = []
text_1 = []

for i, line in enumerate(list_1):
    label_1 = list_1[i][0]
    tex_1 = list_1[i][1:]
    labels_1.append(label_1)
    text_1.append(tex_1)

df = pd.DataFrame(list(zip(labels_1, text_1)),columns =['Reviews', 'Text'])
df.head()

In [ ]:
import nltk
nltk.download()
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


df['cleanText']=df['Text'].map(lambda s:preprocess(s)) 
df.head()

In [ ]:
with open("stsa-test.txt") as txtf:
    test_mylist = [line.rstrip('\n') for line in txtf]
    
test_labels = []
test_text = []

for i, line in enumerate(test_mylist):
    label_test = test_mylist[i][0]
    tex_test = test_mylist[i][1:]
    test_labels.append(label_test)
    test_text.append(tex_test)

df_test = pd.DataFrame(list(zip(test_labels, test_text)),columns =['Reviews', 'Text'])
df_test.head()

In [ ]:
import nltk
nltk.download()
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)


df_test['cleanText']=df_test['Text'].map(lambda s:preprocess(s)) 
df_test.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

tfidf_vectorizer = TfidfVectorizer(lowercase = False, analyzer='word')
train_tfidf = tfidf_vectorizer.fit_transform(df["cleanText"]).toarray()
test_tfidf = tfidf_vectorizer.transform(df_test["cleanText"]).toarray()

In [ ]:
xtest = test_tfidf
ytest = df_test["Reviews"]

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, x_valid, ytrain, y_valid = train_test_split(train_tfidf,df["Reviews"],test_size = 0.2, random_state = 202)

# Algorithms
1. MultinominalNB

In [ ]:
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
model = classifier.fit(xtrain, ytrain) 
predictions_validation_set = classifier.predict(x_valid) 


from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
print ("Accuracy of the Naive Bayes model on validation set is : ", round(accuracy_score(y_valid, predictions_validation_set)*100),"%")
print ("Percision of the Naive Bayes model on validation set is : ", round(precision_score(y_valid, predictions_validation_set, pos_label='0')*100),"%")
print ("Recall of the Naive Bayes model on validation set is : ", round(recall_score(y_valid, predictions_validation_set, pos_label='0')*100),"%")
print ("F1 Score of the Naive Bayes model on validation set is : ", round(f1_score(y_valid, predictions_validation_set, pos_label='0')*100),"%")

In [ ]:
from sklearn.metrics import classification_report

cr_naive_validation = classification_report(y_valid, predictions_validation_set)
print("Classification Report: ", "\n", "\n",cr_naive_validation)

In [ ]:
from sklearn.model_selection import cross_val_score
naive_accuracies_validation = cross_val_score(estimator = classifier, X = xtrain, y = ytrain, cv = 10)

print(f"Naive Bayes Model  10-fold cross validation score on training set is :  {round(naive_accuracies_validation.mean()*100)}%")

In [ ]:
predictions_test_set = classifier.predict(xtest) 
print ("Accuracy of the Naive Bayes model on test set is : ", round(accuracy_score(ytest, predictions_test_set)*100),"%")
print ("Percision of the Naive Bayes model on validation set is : ", round(precision_score(ytest, predictions_test_set, pos_label='0')*100),"%")
print ("Recall of the Naive Bayes model on validation set is : ", round(recall_score(ytest, predictions_test_set, pos_label='0')*100),"%")
print ("F1 Score of the Naive Bayes model on validation set is : ", round(f1_score(ytest, predictions_test_set, pos_label='0')*100),"%")

In [ ]:
cr_naive_test = classification_report(ytest, predictions_test_set)
print("Classification Report: ", "\n", "\n",cr_naive_test)

In [ ]:
naive_accuracies_test = cross_val_score(estimator = classifier, X = xtest, y = ytest, cv = 10)

print(f"Naive Bayes Model 10-fold cross validation score on testing set is :  {round(naive_accuracies_test.mean()*100)}%")

# SVM

In [ ]:
from sklearn import svm

classifier_svm = svm.SVC()
model_svm = classifier_svm.fit(xtrain, ytrain) 
svm_predictions_validation_set = classifier_svm.predict(x_valid) 

print ("Accuracy of the SVM model on validation set is : ", round(accuracy_score(y_valid, svm_predictions_validation_set)*100),"%")
print ("Percision of the SVM model on validation set is : ", round(precision_score(y_valid, svm_predictions_validation_set, pos_label='0')*100),"%")
print ("Recall of the SVM model on validation set is : ", round(recall_score(y_valid, svm_predictions_validation_set, pos_label='0')*100),"%")
print ("F1 Score of the SVM model on validation set is : ", round(f1_score(y_valid, svm_predictions_validation_set, pos_label='0')*100),"%")

In [ ]:
from sklearn.metrics import classification_report

cr_svm_validation = classification_report(y_valid, svm_predictions_validation_set)
print("Classification Report: ", "\n", "\n",cr_svm_validation)

In [ ]:
from sklearn.model_selection import cross_val_score
svm_accuracies_validation = cross_val_score(estimator = classifier_svm, X = xtrain, y = ytrain, cv = 10)

print(f"SVM Model  10-fold cross validation score on training set is :  {round(svm_accuracies_validation.mean()*100)}%")

In [ ]:
svm_predictions_test_set = classifier_svm.predict(xtest) 
print ("Accuracy of the SVM model on test set is : ", round(accuracy_score(ytest, svm_predictions_test_set)*100),"%")
print ("Percision of the SVM model on validation set is : ", round(precision_score(ytest, svm_predictions_test_set, pos_label='0')*100),"%")
print ("Recall of the SVM model on validation set is : ", round(recall_score(ytest, svm_predictions_test_set, pos_label='0')*100),"%")
print ("F1 Score of the SVM model on validation set is : ", round(f1_score(ytest, svm_predictions_test_set, pos_label='0')*100),"%")

In [ ]:
cr_svm_test = classification_report(ytest, svm_predictions_test_set)
print("Classification Report: ", "\n", "\n",cr_svm_test)

In [ ]:
svm_accuracies_test = cross_val_score(estimator = classifier_svm, X = xtest, y = ytest, cv = 10)

print(f"SVM Model 10-fold cross validation score on testing set is :  {round(svm_accuracies_test.mean()*100)}%")

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

classifier_knn = KNeighborsClassifier(n_neighbors = 15)
model_knn = classifier_knn.fit(xtrain, ytrain) 
knn_predictions_validation_set = classifier_knn.predict(x_valid) 

print ("Accuracy of the KNN model on validation set is : ", round(accuracy_score(y_valid, knn_predictions_validation_set)*100),"%")
print ("Percision of the KNN model on validation set is : ", round(precision_score(y_valid, knn_predictions_validation_set, pos_label='0')*100),"%")
print ("Recall of the KNN model on validation set is : ", round(recall_score(y_valid, knn_predictions_validation_set, pos_label='0')*100),"%")
print ("F1 Score of the KNN model on validation set is : ", round(f1_score(y_valid, knn_predictions_validation_set, pos_label='0')*100),"%")

In [ ]:
from sklearn.metrics import classification_report

cr_knn_validation = classification_report(y_valid, knn_predictions_validation_set)
print("Classification Report: ", "\n", "\n",cr_knn_validation)

In [ ]:
from sklearn.model_selection import cross_val_score
knn_accuracies_validation = cross_val_score(estimator = classifier_knn, X = xtrain, y = ytrain, cv = 10)

print(f"KNN Model  10-fold cross validation score on training set is :  {round(knn_accuracies_validation.mean()*100)}%")

In [ ]:
knn_predictions_test_set = classifier_knn.predict(xtest) 
print ("Accuracy of the KNN model on test set is : ", round(accuracy_score(ytest, knn_predictions_test_set)*100),"%")
print ("Percision of the KNN model on validation set is : ", round(precision_score(ytest, knn_predictions_test_set, pos_label='0')*100),"%")
print ("Recall of the KNN model on validation set is : ", round(recall_score(ytest, knn_predictions_test_set, pos_label='0')*100),"%")
print ("F1 Score of the KNN model on validation set is : ", round(f1_score(ytest, knn_predictions_test_set, pos_label='0')*100),"%")

In [ ]:
cr_knn_test = classification_report(ytest, knn_predictions_test_set)
print("Classification Report: ", "\n", "\n",cr_knn_test)

In [ ]:
knn_accuracies_test = cross_val_score(estimator = classifier_knn, X = xtest, y = ytest, cv = 10)

print(f"KNN Model 10-fold cross validation score on testing set is :  {round(knn_accuracies_test.mean()*100)}%")

# Decison Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

classifier_dt = DecisionTreeClassifier()
model_dt = classifier_dt.fit(xtrain, ytrain) 
dt_predictions_validation_set = classifier_dt.predict(x_valid) 

print ("Accuracy of the Decison Tree Classifier model on validation set is : ", round(accuracy_score(y_valid, dt_predictions_validation_set)*100),"%")
print ("Percision of the Decison Tree Classifier model on validation set is : ", round(precision_score(y_valid, dt_predictions_validation_set, pos_label='0')*100),"%")
print ("Recall of the Decison Tree Classifier model on validation set is : ", round(recall_score(y_valid, dt_predictions_validation_set, pos_label='0')*100),"%")
print ("F1 Score of the Decison Tree Classifier model on validation set is : ", round(f1_score(y_valid, dt_predictions_validation_set, pos_label='0')*100),"%")

In [ ]:
from sklearn.metrics import classification_report

cr_dt_validation = classification_report(y_valid, dt_predictions_validation_set)
print("Classification Report: ", "\n", "\n",cr_dt_validation)

In [ ]:
from sklearn.model_selection import cross_val_score
dt_accuracies_validation = cross_val_score(estimator = classifier_dt, X = xtrain, y = ytrain, cv = 10)

print(f"Decison Tree Classifier Model  10-fold cross validation score on training set is :  {round(dt_accuracies_validation.mean()*100)}%")

In [ ]:
dt_predictions_test_set = classifier_dt.predict(xtest) 
print ("Accuracy of the Decison Tree Classifier model on test set is : ", round(accuracy_score(ytest, dt_predictions_test_set)*100),"%")
print ("Percision of the Decison Tree Classifier model on validation set is : ", round(precision_score(ytest, dt_predictions_test_set, pos_label='0')*100),"%")
print ("Recall of the Decison Tree Classifier model on validation set is : ", round(recall_score(ytest, dt_predictions_test_set, pos_label='0')*100),"%")
print ("F1 Score of the Decison Tree Classifier model on validation set is : ", round(f1_score(ytest, dt_predictions_test_set, pos_label='0')*100),"%")

In [ ]:
cr_dt_test = classification_report(ytest, dt_predictions_test_set)
print("Classification Report: ", "\n", "\n",cr_dt_test)

In [ ]:
dt_accuracies_test = cross_val_score(estimator = classifier_dt, X = xtest, y = ytest, cv = 10)

print(f"Decison Tree Classifier Model 10-fold cross validation score on testing set is :  {round(dt_accuracies_test.mean()*100)}%")

# Randomforest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

classifier_rf = RandomForestClassifier()
model_rf = classifier_rf.fit(xtrain, ytrain) 
rf_predictions_validation_set = classifier_rf.predict(x_valid) 

print ("Accuracy of the Random Forest Classifier model on validation set is : ", round(accuracy_score(y_valid, rf_predictions_validation_set)*100),"%")
print ("Percision of the Random Forest Classifier model on validation set is : ", round(precision_score(y_valid, rf_predictions_validation_set, pos_label='0')*100),"%")
print ("Recall of the Random Forest Classifier model on validation set is : ", round(recall_score(y_valid, rf_predictions_validation_set, pos_label='0')*100),"%")
print ("F1 Score of the Random Forest Classifier model on validation set is : ", round(f1_score(y_valid, rf_predictions_validation_set, pos_label='0')*100),"%")

In [ ]:
from sklearn.metrics import classification_report

cr_rf_validation = classification_report(y_valid, rf_predictions_validation_set)
print("Classification Report: ", "\n", "\n",cr_rf_validation)

In [ ]:
from sklearn.model_selection import cross_val_score
rf_accuracies_validation = cross_val_score(estimator = classifier_rf, X = xtrain, y = ytrain, cv = 10)

print(f"Decison Random Forest Model  10-fold cross validation score on training set is :  {round(rf_accuracies_validation.mean()*100)}%")

In [ ]:
rf_predictions_test_set = classifier_rf.predict(xtest) 
print ("Accuracy of the Random Forest Classifier model on test set is : ", round(accuracy_score(ytest, rf_predictions_test_set)*100),"%")
print ("Percision of the Random Forest Classifier model on validation set is : ", round(precision_score(ytest, rf_predictions_test_set, pos_label='0')*100),"%")
print ("Recall of the Random Forest Classifier model on validation set is : ", round(recall_score(ytest, rf_predictions_test_set, pos_label='0')*100),"%")
print ("F1 Score of the Random Forest Classifier model on validation set is : ", round(f1_score(ytest, rf_predictions_test_set, pos_label='0')*100),"%")

In [ ]:
cr_rf_test = classification_report(ytest, rf_predictions_test_set)
print("Classification Report: ", "\n", "\n",cr_rf_test)

In [ ]:
rf_accuracies_test = cross_val_score(estimator = classifier_rf, X = xtest, y = ytest, cv = 10)

print(f"Random Forest Classifier Model 10-fold cross validation score on testing set is :  {round(rf_accuracies_test.mean()*100)}%")

In [ ]:
!pip install xgboost

# XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert y_train and y_valid to integers
ytrain = ytrain.astype(int)
y_valid = y_valid.astype(int)

classifier_xgb = XGBClassifier()
model_xgb = classifier_xgb.fit(xtrain, ytrain) 
xgb_predictions_validation_set = classifier_xgb.predict(x_valid) 

print ("Accuracy of the XGBoost Classifier model on validation set is : ", round(accuracy_score(y_valid, xgb_predictions_validation_set)*100),"%")
print ("Percision of the XGBoost Classifier model on validation set is : ", round(precision_score(y_valid, xgb_predictions_validation_set, pos_label=0)*100),"%")
print ("Recall of the XGBoost Classifier model on validation set is : ", round(recall_score(y_valid, xgb_predictions_validation_set, pos_label=0)*100),"%")
print ("F1 Score of the XGBoost Classifier model on validation set is : ", round(f1_score(y_valid, xgb_predictions_validation_set, pos_label=0)*100),"%")

In [ ]:
from sklearn.metrics import classification_report

cr_xgb_validation = classification_report(y_valid, xgb_predictions_validation_set)
print("Classification Report: ", "\n", "\n",cr_xgb_validation)

In [ ]:
from sklearn.model_selection import cross_val_score
xgb_accuracies_validation = cross_val_score(estimator = classifier_xgb, X = xtrain, y = y_train, cv = 10)

print(f"XGBoost Model  10-fold cross validation score on training set is :  {round(xgb_accuracies_validation.mean()*100)}%")

(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K-means

DBSCAN

Hierarchical clustering

Word2Vec

BERT

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 

# K MEANS

In [ ]:
data_frame = pd.read_csv('Amazon_Unlocked_Mobile.csv')

data_frame['Reviews']=data_frame['Reviews'].map(lambda s:preprocess(s)) 
data_frame.head()

In [ ]:
# TF-IDF VECTORIZATION

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vects = tfidf_vect.fit_transform(data_frame['Reviews'].values.astype('U'))
names= tfidf_vect.get_feature_names()

In [ ]:
## ELBOW METHOD

from sklearn.cluster import KMeans
wcss_1 = []
for i in range(2,12):
    kmeans = KMeans(n_clusters = i, init = "k-means++", random_state = 101)
    kmeans.fit(tfidf_vects)
    wcss_1.append(kmeans.inertia_)

plt.figure(figsize = (11,6))
plt.plot(range(2,12), wcss_1, marker = "o")
plt.title ("The Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("WCSS")

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters = 6,init='k-means++',max_iter=10000, random_state=50)
model.fit(tfidf_vects)
from collections import Counter
Counter(model.labels_)

In [ ]:
top_words_1 = 7
centroids_1 = model.cluster_centers_.argsort()[:, ::-1]
for cluster_num in range(6):
    key_features = [names[i] for i in centroids_1[cluster_num, :top_words_1]]
    print('Cluster '+str(cluster_num+1))
    print('Top Words:', key_features)

In [ ]:
cluster_center=model.cluster_centers_
cluster_center

# DBSCAN

In [ ]:
reviews_1=[]
for i in data_frame['Review']:
    reviews_1.append(str(i).split())
import gensim
w2v_model=gensim.models.Word2Vec(reviews_1, size=100, workers=4)

import numpy as np
vectors_1 = []
for i in reviews_1:
    vector = np.zeros(100)
    counter = 0
    for word in i:
        try:
            vec = w2v_model.wv[word]
            vector += vec
            counter += 1
        except:
            pass
    vector /= counter
    vectors_1.append(vector)  
vectors_1 = np.array(vectors_1)
vectors_1 = np.nan_to_num(vectors_1)

In [ ]:
from sklearn.cluster import DBSCAN
minPts = 2 * 100
# Lower bound function
def lower_bound(nums, target): 
    l, r = 0, len(nums) - 1
    # Binary searching
    while l <= r:
        mid_valueue = int(l + (r - l) / 2)
        if nums[mid_valueue] >= target:
            r = mid_valueue - 1
        else:
            l = mid_valueue + 1
    return l

def compute200thnearestneighbour(x, data): 
    dict_1 = []
    for value in data:
      # computing distances
        dist = np.sum((x - value) **2 ) 
        if(len(dict_1) == 200 and dict_1[199] > dist): 
            l = int(lower_bound(dict_1, dist)) 
            if l < 200 and l >= 0 and dict_1[l] > dist:
                dict_1[l] = dist
        else:
            dict_1.append(dist)
            dict_1.sort()

# Dist 199 contains the distance of 200th nearest neighbour.    
    return dict_1[199]

vectors_1.shape

In [ ]:
# Computing the 200th nearest neighbour distance of some point the dataset:
twohundrethneigh = []
for value in vectors_1[:1000]:
    twohundrethneigh.append( compute200thnearestneighbour(value, vectors_1[:1000]) )
twohundrethneigh.sort()

In [ ]:
# Plotting for the Elbow Method :
%matplotlib inline
from matplotlib import pyplot as plt
plt.figure(figsize=(14,4))
plt.title("Elbow Method for Finding the right Eps hyperparameter")
plt.plot([x for x in range(len(twohundrethneigh))], twohundrethneigh)
plt.xlabel("Number of points")
plt.ylabel("Distance of 200th Nearest Neighbour")
plt.show()

In [ ]:
# Create the model
model_dbs = DBSCAN(eps = 5, min_samples = minPts)
model_dbs.fit(vectors_1)

In [ ]:
df_dbs = data_frame
df_dbs["DBS Cluster Label"] = model_dbs.labels_
df_dbs

# Hierarchical clustering

In [ ]:
import scipy
from scipy.cluster import hierarchy
dendro=hierarchy.dendrogram(hierarchy.linkage(vectors_1,method='ward'))
plt.axhline(y=20)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')  #took n=3 from dendrogram curve 
Agg=cluster.fit_predict(vectors_1)

In [ ]:
data_frame['AVG-W2V Clus Label'] = cluster.labels_
data_frame.head()

In [ ]:
hier_data_frame = data_frame
hier_data_frame["Hierarchial Cluster Labels"] = cluster.labels_
hier_data_frame.groupby(["Hierarchial Cluster Labels"])["Reviews"].count()

In [ ]:
# In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.

In [ ]:
#You can write you answer here. (No code needed)
""""K-means clustering utilizes the concept of measuring the distance between data points and cluster centroids. On the other hand, DBSCAN performs clustering based on the density of data points, detecting areas of high concentration and separating them from sparse regions. Hierarchical clustering operates in a hierarchical manner, initially considering each data point as an individual cluster, and gradually merging the nearest clusters together until all data points belong to a single cluster."""